### pip install & import

In [ ]:
# 설치
# %pip install -r requirements.txt


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# 2003~2023년 데이터 전처리


### 데이터 값 읽어오기

In [ ]:
# 데이터 값 읽어오기 
data_file = "../../data/movie_data/KOBIS_개봉일람_2023-03-23.csv"
df = pd.read_csv(data_file, encoding='UTF-8',index_col = "순번")

### 생략된 내용을 추가로 보는 코드

In [ ]:
# row 생략 없이 출력
pd.set_option('display.max_rows', 10000)
# col 생략 없이 출력
pd.set_option('display.max_columns',10000)

In [ ]:
# 널값 확인
df.isnull()

In [ ]:
# 빈값 확인

df.isnull().sum()

In [ ]:
# 정보 확인
df.info()

### 결측치 데이터 처리하기

In [ ]:
# 제작사, 수입사는 결측치가 너무 많아, drop
# 나머지 결측치는 기타로 변경

df = df.drop(['제작사', '수입사'], axis=1)
df['감독'].fillna('기타', inplace=True)
df['장르'].fillna('기타', inplace=True)
df['배급사'].fillna('기타', inplace=True)
df['영화형태'].fillna('기타', inplace=True)

# 한개가 결측치가 나와서 기타로 치환
df['국적'].fillna('기타', inplace=True)

# 서울 매출액 결측치 제거(사용을 안하니깨문에 0으로 변경)
df['서울 매출액'].fillna(0, inplace=True)
# 남은 결측치 제거

for i in df['개봉일'][df['개봉일'].isnull()].index:
    df = df.drop(i)

for i in df['전국 매출액'][df['전국 매출액'].isnull()].index:
    df = df.drop(i)

for i in df['등급'][df['등급'].isnull()].index:
    df = df.drop(i)

In [ ]:
# 최종확인

df.isnull().sum()

### 범주형 데이터값 확인

In [ ]:
# 범주형 데이터 값 확인

category_cols = ['영화명','감독', '배급사', '영화유형','영화형태','국적','장르','등급','영화구분',]

# 범주형 데이터의 unique 확인
for col in category_cols:
    print('컬럼 [{}] UNIQUE : {}'.format(col, len(df[col].unique())))

### 복수의 감독, 배급사 데이터를 한개 데이터로 변경, (연령)등급 4가지로 정리

In [ ]:
# 다수의 감독 데이터 변경
for idx, row in df.iterrows():
    if ',' in row['감독']:
        directors = row['감독'].split(',')
        # 양 끝 공백 제거 후 정렬
        directors.sort(key=lambda x : x.strip())
        df['감독'][idx] = directors[0].strip()

In [ ]:
# 다수의 배급사를 앞에 있는 배급사로 취급

for idx, row in df.iterrows():
    if ',' in row['배급사']:
        distributor = row['배급사'].split(',')
        # 양 끝 공백 제거 후 변경
        df['배급사'][idx] = distributor[0].strip()

In [ ]:
# 영화유형은 두개 뿐이라 제거

df.drop('영화유형', axis=1, inplace=True)

In [ ]:
# 등급 데이터 전처리

for idx, row in df.iterrows():
    if ',' in row['등급']:
        ratings = row['등급'].split(',')
        # 전체, 12, 15, 청소년관람불가 순
        if '전체' in ratings:
            df['등급'][idx] = '전체관람가'
        elif '12' in ratings:
            df['등급'][idx] = '12세관람가'
        elif '15' in ratings:
            df['등급'][idx] = '15세관람가'
        else:
            df['등급'][idx] = '청소년관람불가'
    elif df['등급'][idx] == '18세관람가' or df['등급'][idx] == '제한상영가': 
        df['등급'][idx] = '청소년관람불가'
    else:
        df['등급'][idx] = df['등급'][idx].replace('이상', '')

### 전국 매출액, 스크린수, 관객수 int 타입변경 

In [ ]:
# 전국 매출액, 스크린 수, 관객수를 확인을 위해 int형으로 변경

df['전국 매출액'] = df['전국 매출액'].str.replace(pat=r',', repl=r'', regex=True)
df['전국 매출액'] = df['전국 매출액'].astype('int64')

df['전국 스크린수'] = df['전국 스크린수'].str.replace(pat=r',', repl=r'', regex=True)
df['전국 스크린수'] = df['전국 스크린수'].astype('int64')

df['전국 관객수'] = df['전국 관객수'].str.replace(pat=r',', repl=r'', regex=True)
df['전국 관객수'] = df['전국 관객수'].astype('int64')


In [ ]:
df.info()

# 연도와 코로나 컬럼을 추가 -> prepricessing의 마지막 단계에 추가 
(1) '연도' 컬럼 integer 타입으로 추가    
(2) '코로나 유무' 컬럼 추가 20,21,22년은 =1, 그 외 연도 = 0 

In [ ]:
# 연도 int로 변경
movie_resize2['연도'] = movie_resize2['연도'].astype('int64')
# 코로나 컬럼 추가 2020~2022년은 1로
movie_resize2['코로나'] = np.where((movie_resize2['연도'].values >=2020) & (movie_resize2['연도'].values <=2022) , 1, 0)

In [ ]:
# 변경 될때만 따로 풀어서 사용하시길.(파일생성코드)
# df.to_csv('./preprocessing.csv',encoding='utf-8')